# Sentiment analysis

##  Set up

In [2]:
%pip install numpy pandas nltk sklearn scipy

Defaulting to user installation because normal site-packages is not writeable
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

In [3]:
import numpy as np 
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

## Load dataset

In [4]:
df = pd.read_csv(r'../resources/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

## Tiền xử lý dữ liệu

### Làm sạch dữ liệu 
- Loại bỏ các thẻ HTML
- Loại bỏ khoảng trắng thừa và dấu câu 
- Loại bỏ stop words
- Chuyển đổi chữ hoa thành chữ thường 
- Chuyển đổi nhãn sang số

#### Loại bỏ thẻ HTML

In [6]:
df['review'] = df['review'].str.replace("<br />", "")
df['review'][2]

'I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.This was the most I\'d laughed at one of Woody\'s comedies in years (dare I say a decade?). While I\'ve never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.'

#### Loại bỏ khoảng trắng thừa và dấu câu

In [7]:
df['review'] = df['review'].str.translate(str.maketrans('', '', string.punctuation))

In [8]:
df['review'][2]

'I thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air conditioned theater and watching a lighthearted comedy The plot is simplistic but the dialogue is witty and the characters are likable even the well bread suspected serial killer While some may be disappointed when they realize this is not Match Point 2 Risk Addiction I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to loveThis was the most Id laughed at one of Woodys comedies in years dare I say a decade While Ive never been impressed with Scarlet Johanson in this she managed to tone down her sexy image and jumped right into a average but spirited young womanThis may not be the crown jewel of his career but it was wittier than Devil Wears Prada and more interesting than Superman a great comedy to go see with friends'

In [9]:
df['review'] = df['review'].str.strip()
df['review'] = df['review'].str.replace(r'\s+',' ', regex=True)
df['review'][2]

'I thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air conditioned theater and watching a lighthearted comedy The plot is simplistic but the dialogue is witty and the characters are likable even the well bread suspected serial killer While some may be disappointed when they realize this is not Match Point 2 Risk Addiction I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to loveThis was the most Id laughed at one of Woodys comedies in years dare I say a decade While Ive never been impressed with Scarlet Johanson in this she managed to tone down her sexy image and jumped right into a average but spirited young womanThis may not be the crown jewel of his career but it was wittier than Devil Wears Prada and more interesting than Superman a great comedy to go see with friends'

#### Loại bỏ stop words

In [10]:
# Tải stop words từ nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [12]:
def remove_stop_words(text):
    words = text.split()
    filter_words = [word for word in words if word not in stop_words]
    return ' '.join(filter_words)

In [13]:
df['review'] = df['review'].apply(remove_stop_words)
df['review'][2]

'I thought wonderful way spend time hot summer weekend sitting air conditioned theater watching lighthearted comedy The plot simplistic dialogue witty characters likable even well bread suspected serial killer While may disappointed realize Match Point 2 Risk Addiction I thought proof Woody Allen still fully control style many us grown loveThis Id laughed one Woodys comedies years dare I say decade While Ive never impressed Scarlet Johanson managed tone sexy image jumped right average spirited young womanThis may crown jewel career wittier Devil Wears Prada interesting Superman great comedy go see friends'

#### Chuyển chữ hoa thành chữ thường 

In [14]:
df['review'] = df['review'].str.lower()
df['review'][2]

'i thought wonderful way spend time hot summer weekend sitting air conditioned theater watching lighthearted comedy the plot simplistic dialogue witty characters likable even well bread suspected serial killer while may disappointed realize match point 2 risk addiction i thought proof woody allen still fully control style many us grown lovethis id laughed one woodys comedies years dare i say decade while ive never impressed scarlet johanson managed tone sexy image jumped right average spirited young womanthis may crown jewel career wittier devil wears prada interesting superman great comedy go see friends'

#### Chuyển đổi nhãn sang số

In [15]:
def sentiment_encode(text):
    if(text == "positive"):
        return 1
    return -1

In [16]:
df['sentiment'] = df['sentiment'].apply(sentiment_encode)

### Chia tập dữ liệu

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

In [18]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.3, random_state=42)

In [19]:
# Chuyển đổi văn bản thành các đặc trưng số sử dụng TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [20]:
X_train_tfidf[0]

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 47 stored elements and shape (1, 5000)>

In [21]:
sentiment = y_train.unique()
sentiment

array([-1,  1])

## Mô hình

### Random forest

In [22]:
# Import các thư viện cần thiết
from sklearn.ensemble import RandomForestClassifier

# Huấn luyện mô hình Random Forest
rf_clf = RandomForestClassifier(
    random_state=42,
)
rf_clf.fit(X_train_tfidf, y_train)


RandomForestClassifier(random_state=42)

In [23]:
# In ra các thông số của mô hình
print(f'n_estimators: {rf_clf.n_estimators}')
print(f'max_depth: {rf_clf.max_depth}')
print(f'min_samples_leaf: {rf_clf.min_samples_leaf}')
print(f'max_features: {rf_clf.max_features}')
print(f'max_samples: {rf_clf.max_samples}')

for id, estimator in enumerate(rf_clf.estimators_):
    print(f'Tree {id}:')
    print(f'  Depth of the tree: {estimator.get_depth()}')
    print(f'  Number of leaves: {estimator.get_n_leaves()}')

depths = [estimator.get_depth() for estimator in rf_clf.estimators_]
n_leaves = [estimator.get_n_leaves() for estimator in rf_clf.estimators_]

print(f'Average depth of the trees: {sum(depths) / len(depths)}')
print(f'Average number of leaves: {sum(n_leaves) / len(n_leaves)}')

n_estimators: 100
max_depth: None
min_samples_leaf: 1
max_features: sqrt
max_samples: None
Tree 0:
  Depth of the tree: 149
  Number of leaves: 5223
Tree 1:
  Depth of the tree: 222
  Number of leaves: 5337
Tree 2:
  Depth of the tree: 202
  Number of leaves: 5331
Tree 3:
  Depth of the tree: 158
  Number of leaves: 5423
Tree 4:
  Depth of the tree: 182
  Number of leaves: 5382
Tree 5:
  Depth of the tree: 189
  Number of leaves: 5371
Tree 6:
  Depth of the tree: 182
  Number of leaves: 5148
Tree 7:
  Depth of the tree: 208
  Number of leaves: 5156
Tree 8:
  Depth of the tree: 193
  Number of leaves: 5246
Tree 9:
  Depth of the tree: 183
  Number of leaves: 5265
Tree 10:
  Depth of the tree: 179
  Number of leaves: 5228
Tree 11:
  Depth of the tree: 184
  Number of leaves: 5223
Tree 12:
  Depth of the tree: 187
  Number of leaves: 4939
Tree 13:
  Depth of the tree: 203
  Number of leaves: 5264
Tree 14:
  Depth of the tree: 231
  Number of leaves: 5293
Tree 15:
  Depth of the tree: 197


In [24]:
#print(f'Depth of the tree: {rf_clf.get_depth()}')
#print(f'Number of leaves: {rf_clf.get_n_leaves()}')
print(f'Number of features: {rf_clf.n_features_in_}')
print(f'Feature importances: {rf_clf.feature_importances_}')

Number of features: 5000
Feature importances: [5.71429951e-04 1.42631322e-04 3.74861192e-04 ... 1.34743421e-04
 1.19586655e-04 4.15686304e-05]


In [25]:
# Dự đoán trên tập train
y_pred_train = rf_clf.predict(X_train_tfidf)

# Đánh giá mô hình
accuracy_train = accuracy_score(y_train, y_pred_train)
report_train = classification_report(y_train, y_pred_train)

print(f'Accuracy: {accuracy_train}')
print('Classification Report:')
print(report_train)

Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00     17589
           1       1.00      1.00      1.00     17411

    accuracy                           1.00     35000
   macro avg       1.00      1.00      1.00     35000
weighted avg       1.00      1.00      1.00     35000



In [26]:
# Dự đoán trên tập test
y_pred = rf_clf.predict(X_test_tfidf)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)



Accuracy: 0.8486666666666667
Classification Report:
              precision    recall  f1-score   support

          -1       0.84      0.86      0.85      7411
           1       0.86      0.84      0.85      7589

    accuracy                           0.85     15000
   macro avg       0.85      0.85      0.85     15000
weighted avg       0.85      0.85      0.85     15000



### Áp dụng phương pháp GridSearch chọn các hyperparameter

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [100, 200, None],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [3, 7, None],
    'max_samples': [0.5, 0.8, None],
}

grid_search = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=param_grid,
    cv=StratifiedKFold(n_splits=5),
)
grid_search.fit(X_train_tfidf, y_train)
print(grid_search.best_params_)

#### Độ chính xác của mô hình với cách chọn tham số Grid Search

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on the small training set using the best estimator
y_pred_train = grid_search.best_estimator_.predict(X_train_tfidf)

# Calculate accuracy and classification report
accuracy_train = accuracy_score(y_train, y_pred_train)
report_train = classification_report(y_train, y_pred_train) 

print(f'Accuracy: {accuracy_train}')
print('Classification Report:')
print(report_train)

Accuracy: 0.5310571428571429
Classification Report:
              precision    recall  f1-score   support

          -1       0.53      0.67      0.59     17589
           1       0.54      0.39      0.45     17411

    accuracy                           0.53     35000
   macro avg       0.53      0.53      0.52     35000
weighted avg       0.53      0.53      0.52     35000



In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on the small test set using the best estimator
y_pred = grid_search.best_estimator_.predict(X_test_tfidf)

# Calculate accuracy and classification report
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.5366
Classification Report:
              precision    recall  f1-score   support

          -1       0.52      0.68      0.59      7411
           1       0.56      0.40      0.46      7589

    accuracy                           0.54     15000
   macro avg       0.54      0.54      0.53     15000
weighted avg       0.54      0.54      0.53     15000



#### Kiểm tra các thuộc tính của cây 

In [ ]:
best_clf = grid_search.best_estimator_

print(f'Depth of the tree: {best_clf.get_depth()}')
print(f'Number of leaves: {best_clf.get_n_leaves()}')
print(f'Number of features: {best_clf.n_features_in_}')
print(f'Feature importances: {best_clf.feature_importances_}')

### Bonus: Train trên dữ liệu nhỏ

In [27]:
import pandas as pd
from scipy.sparse import csr_matrix

# Convert sparse matrices to dense DataFrames
X_train_tfidf_df = pd.DataFrame.sparse.from_spmatrix(X_train_tfidf)
X_test_tfidf_df = pd.DataFrame.sparse.from_spmatrix(X_test_tfidf)

# Reset indices to ensure alignment
X_train_tfidf_df.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_test_tfidf_df.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

# Sample from the dense DataFrames
small_X_train_df = X_train_tfidf_df.sample(n=3500, random_state=42)
small_y_train = y_train.loc[small_X_train_df.index]

small_X_test_df = X_test_tfidf_df.sample(n=1500 , random_state=42)
small_y_test = y_test.loc[small_X_test_df.index]

# Convert back to sparse matrices if needed
small_X_train = csr_matrix(small_X_train_df)
small_X_test = csr_matrix(small_X_test_df)

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

# Handle NaN values in the dataset
imputer = SimpleImputer(strategy='mean')
small_X_train_imputed = imputer.fit_transform(small_X_train)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [100, 200, None],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'max_samples': [0.5, 0.8, None],
}

grid_search = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=param_grid,
    cv=StratifiedKFold(n_splits=5),
)
grid_search.fit(small_X_train_imputed, small_y_train)
print(grid_search.best_params_)

#### Độ chính xác của cách chọn tham số Grid Search trên tập dữ liệu nhỏ

In [29]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on the small training set using the best estimator
small_y_pred_train = grid_search.best_estimator_.predict(small_X_train)

# Calculate accuracy and classification report
accuracy_train = accuracy_score(small_y_train, small_y_pred_train)
report_train = classification_report(small_y_train, small_y_pred_train)

print(f'Accuracy: {accuracy_train}')
print('Classification Report:')
print(report_train)

Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00       512
           1       1.00      1.00      1.00       488

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000



In [30]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on the small test set using the best estimator
small_y_pred = grid_search.best_estimator_.predict(small_X_test)

# Calculate accuracy and classification report
accuracy = accuracy_score(small_y_test, small_y_pred)
report = classification_report(small_y_test, small_y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.8175
Classification Report:
              precision    recall  f1-score   support

          -1       0.82      0.83      0.83       209
           1       0.81      0.80      0.81       191

    accuracy                           0.82       400
   macro avg       0.82      0.82      0.82       400
weighted avg       0.82      0.82      0.82       400

